#Итоговый проект

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from os import listdir
import codecs
import pickle                                                                   
from bs4 import BeautifulSoup as BS
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from boilerpipe.extract import Extractor
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/jpype/_core.py:209: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


In [2]:
train_data = pd.read_csv('train_groups.csv',dtype=np.int16)

In [84]:

test_data= pd.read_csv('test_groups.csv',dtype=np.int16)

In [85]:
#это парсинг теста
import pandas as pd                                                             
import numpy as np                                                              
from tqdm import tqdm                                                           
from os import listdir                                                          
import codecs                                                                   
import pickle                                                                   
from bs4 import BeautifulSoup as BS                                             
from boilerpipe.extract import Extractor                                        
from sys import argv                                                            
                                                                                
parser_type = 'NumWordsRulesExtractor'
def save_obj(obj, name ):                                                       
    with open(name + '.pkl', 'wb') as f:                                        
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)                            
                                                                                
def load_obj(name ):                                                            
    with open(name + '.pkl', 'rb') as f:                                        
        return pickle.load(f)                                                   
                                                                                
path = 'content/'                                                               
texts = {}                                                                      
print('Start: ' + parser_type)                                                  
test_data = pd.read_csv('test_groups.csv',dtype=np.int16)                      
#for filename in ['6770.dat']:                                                  
for filename in tqdm(listdir(path)):                                            
    doc_id = int(filename.strip('.dat'))                                        
    if doc_id not in test_data.doc_id.values:                                  
        continue                                                                
    try:                                                                        
        with codecs.open(path + filename, 'r', encoding='utf-8', errors='ignore') as f:
            url = f.readline().strip()                                          
            html = f.read()                                                     
            extractor = Extractor(extractor=parser_type, html=html)             
            s = extractor.getText()                                             
            s=s.replace('\n'," ")                                               
            s=s.replace('\t'," ")                                               
            s=s.replace('\r'," ")                                               
            texts[doc_id] = s                                                   
    except UnicodeDecodeError: 
        print('error {}'.format(filename))                                      
        with codecs.open(path + filename, 'r', 'utf-8') as f:                   
            url = f.readline().strip()                                          
            html = f.read()                                                     
        bs = BS(html, 'html.parser')#ищем след страницу                         
        for s in bs.select('script'):                                           
            s.extract()                                                         
        s = bs.get_text()                                                       
        s=s.replace('\n',"")                                                    
        s=s.replace('\t',"")                                                    
        s=s.replace('\r',"")                                                    
        texts[doc_id] = s                                                       
test_data['text'] = test_data.apply(lambda row: texts[row.doc_id], axis = 1)  


Start: NumWordsRulesExtractor


 47%|████▋     | 13063/28026 [10:07<17:59, 13.86it/s]  

error 6770.dat


100%|██████████| 28026/28026 [20:44<00:00, 22.53it/s]


In [3]:
def save_obj(obj, name ):                                                       
    with open(name + '.pkl', 'wb') as f:                                        
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)                            
                                                                                
def load_obj(name ):                                                            
    with open(name + '.pkl', 'rb') as f:                                        
        return pickle.load(f)       

In [4]:
#START HEAR

In [5]:
name = 'NumWordsRulesExtractor'
name = 'train_data'+name

In [6]:
train_data = load_obj(name)#Загружаем готовый распарсенный текст уже

In [86]:
doc_to_title = {}
with open('docs_titles.tsv', encoding = 'UTF-8') as f:
    for num_line, line in enumerate(f):
        if num_line == 0:
            continue
        data = line.strip().split('\t', 1)
        doc_id = int(data[0])
        if len(data) == 1:
            title = ''
        else:
            title = data[1]
        doc_to_title[doc_id] = title
print (len(doc_to_title))


28026


In [88]:
doc_titles = []
for id in train_data.doc_id.values:
    doc_titles.append(doc_to_title[id])

In [89]:
train_data['title'] = doc_titles

In [87]:
test_doc_titles = []
for id in test_data.doc_id.values:
    test_doc_titles.append(doc_to_title[id])
test_data['title'] = test_doc_titles

In [90]:
def get_splits(data,n=3):
    bound =[]
    max_group = data.group_id.max()
    bound.extend([i*int(max_group/n)+1 for i in range(1,n)])
    bound.append(max_group+1)
    prev_id = 1
    for group_id in bound:
        yield (data[~((prev_id <= data.group_id) & (data.group_id < group_id))].pair_id.values-1,
            data[(prev_id <= data.group_id) & (data.group_id < group_id)].pair_id.values-1)
        prev_id = group_id

In [106]:
vectorizer = CountVectorizer()#max_features = 70)
#texts_vectorized = vectorizer.fit_transform(train_data.text)

In [107]:
test_data.group_id.values

array([130, 130, 130, ..., 309, 309, 309], dtype=int16)

In [93]:
groups_tmp = np.concatenate((train_data.group_id.values,  test_data.group_id.values))

In [108]:
gr = np.array(list(range(1,310))).reshape(-1,1)
gr

array([[  1],
       [  2],
       [  3],
       [  4],
       [  5],
       [  6],
       [  7],
       [  8],
       [  9],
       [ 10],
       [ 11],
       [ 12],
       [ 13],
       [ 14],
       [ 15],
       [ 16],
       [ 17],
       [ 18],
       [ 19],
       [ 20],
       [ 21],
       [ 22],
       [ 23],
       [ 24],
       [ 25],
       [ 26],
       [ 27],
       [ 28],
       [ 29],
       [ 30],
       [ 31],
       [ 32],
       [ 33],
       [ 34],
       [ 35],
       [ 36],
       [ 37],
       [ 38],
       [ 39],
       [ 40],
       [ 41],
       [ 42],
       [ 43],
       [ 44],
       [ 45],
       [ 46],
       [ 47],
       [ 48],
       [ 49],
       [ 50],
       [ 51],
       [ 52],
       [ 53],
       [ 54],
       [ 55],
       [ 56],
       [ 57],
       [ 58],
       [ 59],
       [ 60],
       [ 61],
       [ 62],
       [ 63],
       [ 64],
       [ 65],
       [ 66],
       [ 67],
       [ 68],
       [ 69],
       [ 70],
       [ 71],
      

In [109]:
from sklearn.preprocessing import OneHotEncoder
i = OneHotEncoder()
i.fit(gr)
groups_one_hot = i.transform(test_data.group_id.values.reshape(-1,1))
groups_one_hot 

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


<16627x309 sparse matrix of type '<class 'numpy.float64'>'
	with 16627 stored elements in Compressed Sparse Row format>

In [110]:
groups_one_hot

<16627x309 sparse matrix of type '<class 'numpy.float64'>'
	with 16627 stored elements in Compressed Sparse Row format>

In [96]:
#texts_vectorized.shape

In [111]:
from scipy.sparse import hstack
from scipy.sparse import vstack

In [112]:
#features = hstack( [texts_vectorized,groups_one_hot] )

In [75]:
target = train_data.target.values

In [76]:
#doc_id = train_data.doc_id.values

In [113]:
groups = test_data.group_id.values

In [114]:
groups

array([130, 130, 130, ..., 309, 309, 309], dtype=int16)

In [115]:
data = test_data

In [116]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import pairwise_distances
scaler = StandardScaler(with_mean=False)
number = 1
for group in tqdm(set(groups)):
    number +=1
    elements_in_same_group = data.group_id == group
#    print(elements_in_same_group)
    vectorizer.fit(data.title[elements_in_same_group] +data.text[elements_in_same_group])
    texts_vectorized_titles = vectorizer.transform(data.title[elements_in_same_group])
    texts_vectorized_titles = scaler.fit_transform(texts_vectorized_titles)
    
    texts_vectorized_texts = vectorizer.transform(data.text[elements_in_same_group])
    texts_vectorized_texts = scaler.fit_transform(texts_vectorized_texts)

    #dist_matr = pairwise_distances(texts_vectorized.tocsr()[elements_in_same_group], texts_vectorized.tocsr()[elements_in_same_group], metric = 'cosine')
    dist_matr = pairwise_distances(texts_vectorized_titles, texts_vectorized_titles, metric = 'cosine')
    dist_matr2 = pairwise_distances(texts_vectorized_texts, texts_vectorized_texts, metric = 'cosine')
    dist_matr3 = pairwise_distances(texts_vectorized_texts, texts_vectorized_titles, metric = 'cosine')
    #print(texts_vectorized.shape)
    #clustering = DBSCAN(eps = 0.5, min_samples=3).fit(dist_matr)
    dist_matr_sorted = np.sort(dist_matr,axis=1)[:,0:21][:,::-1]
    dist_matr_sorted2 = np.sort(dist_matr2,axis=1)[:,0:21][:,::-1]
    dist_matr_sorted3 = np.sort(dist_matr3,axis=1)[:,0:21][:,::-1]
    mean_dist = dist_matr.mean(axis = 1)
    dist_matr_sorted = np.hstack((dist_matr_sorted, mean_dist.reshape(-1,1)))
    dist_matr_sorted = np.hstack((dist_matr_sorted, dist_matr_sorted2))
    dist_matr_sorted = np.hstack((dist_matr_sorted, dist_matr_sorted3))

#    dist_matr_sorted = hstack((dist_matr_sorted, texts_vectorized))
    
    if number == 2 :
        features_by_dist =  dist_matr_sorted
    else:
        features_by_dist = vstack((features_by_dist, dist_matr_sorted))

100%|██████████| 180/180 [05:08<00:00,  1.72s/it]


In [117]:
features_by_dist = hstack([features_by_dist, groups_one_hot])

In [118]:
features_by_dist.shape

(16627, 373)

In [104]:
#SAVE model

from sklearn.ensemble import RandomForestClassifier 
model = RandomForestClassifier(n_jobs=6,n_estimators=200)
model.fit(features_by_dist.tocsr(), target)
save_obj(model, 'model')

In [105]:
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier 
max_score = 0
scores = []
for train, test in get_splits(train_data,n=3):
            print(type(features_by_dist))
            model = RandomForestClassifier(n_jobs=6,n_estimators=200)
            model.fit(features_by_dist.tocsr()[train], target[train])
            pred = (model.predict(features_by_dist.tocsr()[test]))               
            scores.append(f1_score(pred,target[test]))
            print(scores)
print(sum(scores)/3)

<class 'scipy.sparse.coo.coo_matrix'>
[0.7113783533765031]
<class 'scipy.sparse.coo.coo_matrix'>
[0.7113783533765031, 0.7077534791252486]
<class 'scipy.sparse.coo.coo_matrix'>
[0.7113783533765031, 0.7077534791252486, 0.7351885098743268]
0.7181067807920262


In [119]:
#SUBMIT
from sklearn.ensemble import RandomForestClassifier 
print(type(features_by_dist))
model = load_obj('model') 
pred = (model.predict(features_by_dist.tocsr()))               

<class 'scipy.sparse.coo.coo_matrix'>


In [120]:
pred.shape

(16627,)

In [122]:
test_data['target'] = pred

In [127]:
test_data.to_csv('sub',index=False,columns=['pair_id','target'])